In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from scipy import stats
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
!pip install xlsxwriter
pd.options.mode.chained_assignment = None  # default='warn'
import statsmodels.api as sm
from statsmodels.formula.api import ols
## https://www.kaggle.com/code/alexmaszanski/two-way-anova-with-python/notebook
from scipy.stats import linregress
import glob
import researchpy as rp
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import re
from statsmodels.stats.anova import AnovaRM
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


**There are four assumptions that must be met before using two-way ANOVA:**

- Normality: Observations from the sample population are normally distributed.
- Sample Size: The number of observations must be the same for each group.
- Equal Variances: The variances for each group are equal.
- Independence: Observations in each group are independent.

In [2]:
currdir= os.getcwd()
parent = os.path.dirname(currdir)
gparent=os.path.dirname(parent)
lines_to_skip = 10 # adjust this as necessary

# count the number of header lines
header_lines = 3 # adjust this as necessary

plate_type = 'DIV19'
# read the csv file into a pandas DataFrame, skipping the metadata at the top
# df = pd.read_csv(filename, skiprows=lines_to_skip, header=[i for i in range(header_lines)])
### read all data

PERTURBATION = "CNO"
firstTableHeading = "Number of Spikes" #"Mean Firing Rate (Hz)"
basalname = "basal"
largedosename = "20uM_CNO"
## conditions taken: ### we will define this as dose1,dose2, dose3, dose4, each incrementing in terms of doses
basal_folder = f"{parent}/data/DIV19_{PERTURBATION}/Div19_basal_CNO"
dose1_folder =  f"{parent}/data/DIV19_{PERTURBATION}/Div19_20uM_CNO"


doses= {"basal": basalname,
        "dose1":largedosename }
basal_folder

'/home/poojaparameswaran/Documents/SoderlingLab/MEA_analysis/data/DIV19_CNO/Div19_basal_CNO'

In [3]:
def read_csvs_to_dict(folder):
    files = glob.glob(os.path.join(folder, "*.csv"))
    data_dict = {}
    for file in files:
        # Extracting the plate number from the filename
        plate_num = re.search(r'plate\s?(\d+)', file, re.IGNORECASE)
        if plate_num:
            plate_key = f"plate_{plate_num.group(1)}"
            df = pd.read_csv(file, skiprows = lines_to_skip)
            df = df.drop(columns=['Unnamed: 9'])
            data_dict[plate_key] = df
    return data_dict
basal_files_dict = read_csvs_to_dict(basal_folder)
dose1_files_dict = read_csvs_to_dict(dose1_folder)

for ind, dictionary in enumerate([basal_files_dict, dose1_files_dict]):
    print(list(doses.keys())[ind], dictionary.keys() )

basal dict_keys(['plate_3', 'plate_1'])
dose1 dict_keys(['plate_3', 'plate_1'])


In [6]:
def concat_plates(dfs_dict, firstHeading):
    experiment = {}
    testparams_concat = {}
    for plate, df in dfs_dict.items():
        maxrows = df.shape[0]
        vals_between_tables = 16
        titles = [heading_ind for heading_ind in range(14, maxrows+16, 16)]
        titles.insert(0,0)
        # # Empty dictionary to store dataframes
        # Loop through start indices
        for ind in range(len(titles) -1):
            if ind == 0:
                ## Handle mean firing rate
                key = firstHeading
                table = df.iloc[titles[ind] : titles[ind +1], :]
                # table = rename_row_names(table)
            elif ind != 0:
                table = df.iloc[titles[ind] : titles[ind +1], :]
                key = table.iloc[0][0] # just get name
                table = df.iloc[titles[ind] +2: titles[ind +1], :]

            table.set_index('Unnamed: 0', inplace=True)
            if key not in experiment:
                experiment[key] = []
            experiment[key].append(table)
    
    for testparam, df_list in experiment.items():
        concated_table = pd.concat(df_list, ignore_index=False)
        testparams_concat[testparam] = concated_table
    
    testparams_concat = {key.strip(): value for key, value in testparams_concat.items()}
    return testparams_concat

basal_dict_wmeanSEM= concat_plates(basal_files_dict, firstTableHeading)
dose1_dict_wmeanSEM = concat_plates(dose1_files_dict, firstTableHeading)
dose1_dict_wmeanSEM['Number of Spikes']

,GPR37L 1,GPR37L 2,LGI2 A12,SLITRK5,THSD7 1,THSD7 2,unt1,unt2
Unnamed: 0,,,,,,,,
B Mean,16361.166667,20843.083333,18982.416667,10884.250000,14579.333333,14201.250000,15611.833333,15437.666667
B SEM,2460.238680,2772.304157,1981.345397,1567.180034,2365.293612,2481.827400,1879.527894,2981.369808
B Replicates,20349.000000,20101.000000,17877.000000,808.000000,19416.000000,12501.000000,6353.000000,15061.000000
NaN,7825.000000,15735.000000,10794.000000,7501.000000,6522.000000,12496.000000,13301.000000,7013.000000
NaN,7249.000000,16891.000000,20627.000000,6401.000000,11652.000000,2605.000000,23365.000000,10761.000000
NaN,6684.000000,12922.000000,8705.000000,15806.000000,5332.000000,4307.000000,22297.000000,8611.000000
NaN,16165.000000,25412.000000,26219.000000,15957.000000,7181.000000,5748.000000,12425.000000,6665.000000
NaN,5459.000000,8923.000000,13542.000000,12495.000000,15956.000000,14485.000000,8268.000000,16226.000000
NaN,15406.000000,21965.000000,14577.000000,16456.000000,14024.000000,16379.000000,20082.000000,14134.000000


In [34]:
def remove_mean_SEM(dictionary, chemical_type):
    for key, value in dictionary.items():
        to_drop = [index for index in ['B Mean', 'B SEM'] if index in value.index]
        if to_drop:
            dictionary[key] = value.drop(index=to_drop)
        dictionary[key]['Chemical_Type']=chemical_type
    return dictionary

basal_dictmessy = remove_mean_SEM(basal_dict_wmeanSEM, chemical_type="basal")
dose1_dictmessy = remove_mean_SEM(dose1_dict_wmeanSEM,chemical_type=largedosename)


# print(basal_dict['Number of Covered Electrodes'].columns)
# dose1_dictmessy['Mean Firing Rate (Hz)']
dose1_dictmessy.keys()


dict_keys(['Number of Spikes', 'Mean Firing Rate (Hz)', 'ISI Coefficient of Variation - Avg', 'Network ISI Coefficient of Variation', 'Fano Factor', 'Number of Active Electrodes', 'Weighted Mean Firing Rate (Hz)', 'Number of Bursts', 'Number of Bursting Electrodes', 'Burst Duration - Avg (sec)', 'Burst Duration - Std (sec)', 'Burst Duration (Median) - Avg (sec)', 'Burst Duration (Median) - Std (sec)', 'Number of Spikes per Burst - Avg', 'Number of Spikes per Burst - Std', 'Number of Spikes per Burst (Median) - Avg', 'Number of Spikes per Burst (Median) - Std', 'Mean ISI within Burst - Avg (sec)', 'Mean ISI within Burst - Std (sec)', 'Median ISI within Burst - Avg (sec)', 'Median ISI within Burst - Std (sec)', 'Median/Mean ISI within Burst - Avg', 'Median/Mean ISI within Burst - Std', 'Inter-Burst Interval - Avg (sec)', 'Inter-Burst Interval - Std (sec)', 'Inter-Burst Interval (Median) - Avg (sec)', 'Inter-Burst Interval (Median) - Std (sec)', 'Burst Frequency - Avg (Hz)', 'Burst Freque

In [35]:
def rename_row_names(experiment_df):
    # 'Unnamed: 0' is a column
    experiment_df = experiment_df.reset_index()
    experiment_df = experiment_df.drop(columns="Unnamed: 0", errors="ignore")
    experiment_df.index = ["Replicate{}".format(i+1) for i in range(len(experiment_df))]
    return experiment_df

basal_dict = {key: rename_row_names(df) for key, df in basal_dictmessy.items()}
dose1_dict = {key: rename_row_names(df) for key, df in dose1_dictmessy.items()}


In [36]:
all_dicts = dict(zip(doses.keys(),[basal_dict, dose1_dict]))
all_dicts['basal']['Mean Firing Rate (Hz)']

,GPR37L 1,GPR37L 2,LGI2 A12,SLITRK5,THSD7 1,THSD7 2,unt1,unt2,Chemical_Type
Replicate1,1.998131,2.480482,3.545681,0.635797,4.162791,3.440822,2.032600,3.481935,basal
Replicate2,0.106312,0.941860,1.087002,1.029693,0.445390,0.392234,1.375208,0.515158,basal
Replicate3,0.438953,2.119394,1.528654,1.060631,0.283638,0.159468,2.204527,1.246885,basal
Replicate4,0.313538,0.938331,0.412583,1.411130,0.299834,0.127284,2.550249,0.411337,basal
Replicate5,0.409676,0.872716,1.966362,1.630606,0.873131,0.536752,1.439784,1.621885,basal
Replicate6,0.413621,0.329111,0.524709,0.968023,1.083472,1.512251,1.254776,2.459095,basal
Replicate7,0.848837,0.655731,1.789037,1.564784,1.218439,0.774709,1.980897,0.884551,basal
Replicate8,1.208264,1.370640,1.837209,1.511005,0.846346,0.554194,1.269726,2.671512,basal
Replicate9,1.439369,2.109842,1.538206,2.290490,1.908015,2.876869,1.673588,5.962832,basal
Replicate10,1.073297,1.202243,2.366902,1.546719,4.039244,2.847799,2.359635,2.512043,basal


In [37]:
def combine_chem_types(dict_of_alldicts):
    all_concat = {key: None for key in dict_of_alldicts['basal']}
    for type in dict_of_alldicts.keys():
        for test_parameter, df in dict_of_alldicts[type].items():
            all_concat[test_parameter]=pd.concat([all_concat[test_parameter], df], ignore_index=False)
    for testparam, df in all_concat.items():
        df.columns = df.columns.str.replace(' ', '_')
        for gene in df.columns:
            if gene != "Chemical_Type":
                df[gene]= df[gene].astype(float)
    return all_concat

TestParams_dict = combine_chem_types(all_dicts)
print(TestParams_dict['Network Burst Duration - Avg (sec)']['Chemical_Type'].unique())
TestParams_dict['Network Burst Duration - Avg (sec)']

['basal' '20uM_CNO']


,GPR37L_1,GPR37L_2,LGI2_A12,SLITRK5,THSD7_1,THSD7_2,unt1,unt2,Chemical_Type
Replicate1,5.2e-01,2.8e-01,3.0e-01,1.5e-01,3.1e-01,6.3e-01,1.3e-01,4.4e-01,basal
Replicate2,4.6e-01,6.1e-01,4.1e-01,2.1e-01,8.2e-01,6.1e-01,6.2e-01,4.9e-01,basal
Replicate3,4.8e-01,3.1e-01,3.3e-01,3.1e-01,3.3e-01,4.7e-01,3.5e-01,1.0e+00,basal
Replicate4,2.7e-01,2.9e-01,4.4e-01,2.7e-01,2.4e-01,4.9e-01,4.7e-01,5.9e-01,basal
Replicate5,3.5e-01,6.9e-01,3.9e-01,3.8e-01,3.1e-01,2.9e-01,3.9e-01,3.4e-01,basal
Replicate6,6.7e-01,5.0e-01,4.6e-01,2.4e-01,4.4e-01,3.6e-01,6.2e-01,1.6e+00,basal
Replicate7,3.7e-01,4.8e-01,5.2e-01,3.9e-01,3.6e-01,2.7e-01,3.4e-01,4.4e-01,basal
Replicate8,7.1e-01,4.6e-01,4.3e-01,8.7e-01,4.0e-01,5.3e-01,3.9e-01,5.0e-01,basal
Replicate9,3.5e-01,5.4e-01,7.9e-01,5.9e-01,7.8e-01,2.4e-01,2.9e-01,3.4e-01,basal
Replicate10,2.9e-01,3.8e-01,4.8e-01,7.4e-01,5.0e-01,8.3e-01,2.7e-01,4.1e-01,basal


In [38]:
TestParams_dict['Synchrony Index']

,GPR37L_1,GPR37L_2,LGI2_A12,SLITRK5,THSD7_1,THSD7_2,unt1,unt2,Chemical_Type
Replicate1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,basal
Replicate2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,basal
Replicate3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,basal
Replicate4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,basal
Replicate5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,basal
Replicate6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,basal
Replicate7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,basal
Replicate8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,basal
Replicate9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,basal
Replicate10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,basal


In [39]:
def impute_nans(testparam, table):
    chem_types = table['Chemical_Type']
    table = table.drop(columns=['Chemical_Type'])
    original_index = table.index
    imputer = IterativeImputer(max_iter=100, random_state=0)
    imputed_data = imputer.fit_transform(table)

    # Convert the imputed data back to a DataFrame
    imputed_df = pd.DataFrame(imputed_data, columns=table.columns, index=original_index)
    imputed_df['Chemical_Type'] = chem_types
    return imputed_df

TestParams_dict_imputed = {test_param: table for test_param, table in TestParams_dict.items()}
for key, table in TestParams_dict.items():
    if table.loc[:, table.columns != 'Chemical_Type'].isna().all().all():
        del TestParams_dict_imputed[key]
    else:
        TestParams_dict_imputed[key] = impute_nans(key, table)

/home/poojaparameswaran/anaconda3/envs/dataenv/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/home/poojaparameswaran/anaconda3/envs/dataenv/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/home/poojaparameswaran/anaconda3/envs/dataenv/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [40]:
def melt_df_Long(df):
    new_df = pd.melt(df, id_vars=['Chemical_Type'], var_name='Gene', value_name='Value')
    return new_df

for test_param, table in TestParams_dict_imputed.items():
    TestParams_dict_imputed[test_param] = melt_df_Long(table)

In [41]:
TestParams_dict_imputed['LFP Theta (4-8 Hz) Relative Power - Avg']['Gene'].unique()

array(['GPR37L_1', 'GPR37L_2', 'LGI2_A12', 'SLITRK5', 'THSD7_1',
       'THSD7_2', 'unt1', 'unt2'], dtype=object)

In [42]:
def categorize_gene_vs_control(testparam_df, control_var='unt2'):
    genes = list(testparam_df['Gene'].unique())
    mutants = [gene for gene in genes if "unt" not in gene]
    mutVcont_dict= {mut: pd.DataFrame for mut in mutants}
    for mutant in mutants:
        tiny_df = testparam_df[(testparam_df['Gene'] == control_var) |\
                   (testparam_df['Gene'] == mutant)]
        mutVcont_dict[mutant] = tiny_df
    return mutVcont_dict

TestParamsSectioned_dict = TestParams_dict_imputed.copy()

for test_param, table in TestParams_dict_imputed.items():
    TestParamsSectioned_dict[test_param] = categorize_gene_vs_control(table, control_var='unt2')

TestParamsSectioned_dict['Number of Active Electrodes']

{'GPR37L_1':     Chemical_Type      Gene   Value
 0           basal  GPR37L_1 8.0e+00
 1           basal  GPR37L_1 6.0e+00
 2           basal  GPR37L_1 8.0e+00
 3           basal  GPR37L_1 8.0e+00
 4           basal  GPR37L_1 8.0e+00
 ..            ...       ...     ...
 379      20uM_CNO      unt2 8.0e+00
 380      20uM_CNO      unt2 8.0e+00
 381      20uM_CNO      unt2 8.0e+00
 382      20uM_CNO      unt2 8.0e+00
 383      20uM_CNO      unt2 7.0e+00
 
 [96 rows x 3 columns],
 'GPR37L_2':     Chemical_Type      Gene   Value
 48          basal  GPR37L_2 8.0e+00
 49          basal  GPR37L_2 8.0e+00
 50          basal  GPR37L_2 7.0e+00
 51          basal  GPR37L_2 8.0e+00
 52          basal  GPR37L_2 7.0e+00
 ..            ...       ...     ...
 379      20uM_CNO      unt2 8.0e+00
 380      20uM_CNO      unt2 8.0e+00
 381      20uM_CNO      unt2 8.0e+00
 382      20uM_CNO      unt2 8.0e+00
 383      20uM_CNO      unt2 7.0e+00
 
 [96 rows x 3 columns],
 'LGI2_A12':     Chemical_Type      

In [43]:
### You want to troubleshoot w PRISM??

TESTPARAM = 'Area Under Normalized Cross-Correlation'
GENEINT = 'GPR37L_1'
# TestParamsSectioned_dict[TESTPARAM][GENEINT].to_csv(f"{parent}/data/PRISM_inputs/{TESTPARAM}_{GENEINT}.csv")

In [44]:
TestParamsSectioned_dict[TESTPARAM][GENEINT]

,Chemical_Type,Gene,Value
0,basal,GPR37L_1,8.8e-01
1,basal,GPR37L_1,7.1e-01
2,basal,GPR37L_1,6.8e-01
3,basal,GPR37L_1,7.1e-01
4,basal,GPR37L_1,6.8e-01
...,...,...,...
379,20uM_CNO,unt2,6.7e-01
380,20uM_CNO,unt2,8.1e-01
381,20uM_CNO,unt2,7.9e-01
382,20uM_CNO,unt2,8.0e-01


In [45]:
def ensure_dirs_exists(path):
    directory = os.path.dirname(path)  # Get the directory part of the file path
    if not os.path.exists(directory):
        os.makedirs(directory)
    return

In [46]:

def calculate_ANOVA(testparameter, df, mutant_gene, anovaRM=True,
                    show_barlabels = False, control_var="unt2"):
    """_summary_
    In a two-way repeated anova measurement test, 1 factor== within-subject, each 
        subject is exposed to all levels of this factor (chemical_Type).
        -Value is dependent variable
        -Gene is index, each gene is eitehr control or mutation so it is key identifier
        -Chemical_type is change within each subject
    
    Args:
        testparameter (_type_): _description_from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

    Returns:
        Anova TABLE: showing mut (1) vs control (1), significance of gene
                        perturbation, gene&chemical
                        
            sum_sq: measure of total variation within data. how much data points differ from the mean
            df (deg of freedom): independ. values, values that are free to vary once we know
                            constraints (sample means)
            F-statistic:  model variance / residual variance. amount of variance due to the variable.
            P-value: is the variance noted (sum_sq, F-statistic) significant?
    """
    ## Flag vals that have been imputed?
    # Impute missing values using IterativeImputer
    if anovaRM:
        # print(df.shape, df.head())
        df = df.groupby(['Gene', 'Chemical_Type']).mean().reset_index()
        anova_table = AnovaRM(df, depvar='Value', subject='Gene', within=['Chemical_Type']).fit()
        print(anova_table)
    else:
        model = ols('Value ~ C(Gene) + C(Chemical_Type) + C(Gene):C(Chemical_Type)', data= df).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)
        anova_table = anova_table.drop('Residual')
        # anova_table = anova_table.rename(index={'C(Gene)': f'C({mutant_gene})'})
    return anova_table

ANOVA_dict = {test_param: {} for test_param, dictionary in TestParamsSectioned_dict.items()}
for test_param, mutVcontrol_dict in TestParamsSectioned_dict.items():
    for mut, table in mutVcontrol_dict.items():
            if table['Chemical_Type'].nunique() > 1:
                ANOVA_dict[test_param][mut]= calculate_ANOVA(test_param, table, mut, anovaRM=False)

# ANOVA_dict['Mean Firing Rate (Hz)']['GPR37L_1']

/home/poojaparameswaran/anaconda3/envs/dataenv/lib/python3.11/site-packages/statsmodels/base/model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  warnings.warn('covariance of constraints does not have full '
/home/poojaparameswaran/anaconda3/envs/dataenv/lib/python3.11/site-packages/statsmodels/base/model.py:1917: RuntimeWarning: invalid value encountered in divide
  F /= J
/home/poojaparameswaran/anaconda3/envs/dataenv/lib/python3.11/site-packages/statsmodels/base/model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  warnings.warn('covariance of constraints does not have full '
/home/poojaparameswaran/anaconda3/envs/dataenv/lib/python3.11/site-packages/statsmodels/base/model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  warnings.warn('covariance of constraints doe

In [47]:
ANOVA_dict

{'Number of Spikes': {'GPR37L_1':                           sum_sq      df       F  PR(>F)
  C(Gene)                  1.3e+07 1.0e+00 1.3e-01 7.2e-01
  C(Chemical_Type)         3.1e+08 1.0e+00 3.1e+00 8.4e-02
  C(Gene):C(Chemical_Type) 1.4e+08 1.0e+00 1.4e+00 2.3e-01,
  'GPR37L_2':                           sum_sq      df       F  PR(>F)
  C(Gene)                  3.3e+08 1.0e+00 3.2e+00 7.8e-02
  C(Chemical_Type)         5.5e+08 1.0e+00 5.3e+00 2.4e-02
  C(Gene):C(Chemical_Type) 3.2e+08 1.0e+00 3.1e+00 8.2e-02,
  'LGI2_A12':                           sum_sq      df       F  PR(>F)
  C(Gene)                  2.7e+04 1.0e+00 3.4e-04 9.9e-01
  C(Chemical_Type)         6.7e+08 1.0e+00 8.4e+00 4.7e-03
  C(Gene):C(Chemical_Type) 4.1e+08 1.0e+00 5.2e+00 2.5e-02,
  'SLITRK5':                           sum_sq      df       F  PR(>F)
  C(Gene)                  1.6e+08 1.0e+00 2.3e+00 1.3e-01
  C(Chemical_Type)         5.2e+07 1.0e+00 7.5e-01 3.9e-01
  C(Gene):C(Chemical_Type) 2.8e+06 1.0e+00 4.

In [48]:
def create_multind_df(ANOVA_dict):
    pd.set_option('display.float_format', lambda x: '%.1e' % x)
    test_params = list(ANOVA_dict.keys())
    mutant_genes = list(ANOVA_dict[test_params[0]].keys())
    anova_variables = list(ANOVA_dict[test_params[0]][mutant_genes[0]].index)
    
    col_iterables = [mutant_genes, ["F-statistic", "P-value"]]
    row_iterables = [test_params, anova_variables]
    
    rindex = pd.MultiIndex.from_product(row_iterables, names=["Test Parameter", "ANOVA variables"])
    cindex = pd.MultiIndex.from_product(col_iterables)
    
    df = pd.DataFrame(index=rindex, columns= cindex)
    for test_param, testparam_dict in ANOVA_dict.items():
        for mutant, anova_df in testparam_dict.items():
            for var in anova_variables:
                df.loc[(test_param, var), (mutant, 'F-statistic')] = anova_df.loc[var, 'F']
                df.loc[(test_param, var), (mutant, 'P-value')] = anova_df.loc[var, 'PR(>F)']
                # df.loc[(test_param, 'C(Gene)'), (mutant, 'F-statistic')] = (anova_df['F']['C(Gene)'])
    return df

nf= create_multind_df(ANOVA_dict)
SAVE_ANOVA_TABLE_FILE = f"{parent}/twoWayANOVA_tables/{PERTURBATION}/{plate_type}_{PERTURBATION}_ANOVAresults_basal_MAXDOSE.csv"
ensure_dirs_exists(SAVE_ANOVA_TABLE_FILE)
nf.to_csv(SAVE_ANOVA_TABLE_FILE)

In [49]:
from openpyxl import Workbook
from openpyxl.styles import PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows
def color_p_values(df, output_file_name):
    """
    Apply red color formatting to cells in 'P-value' columns where the value is less than 0.05.

    Parameters:
    df (pandas.DataFrame): The dataframe to process.
    output_file_name (str): The name of the output Excel file.
    """
    # Create a new workbook and select the active worksheet
    wb = Workbook()
    sheet = wb.active

    # Convert the DataFrame to rows and add them to the sheet
    for r_idx, row in enumerate(dataframe_to_rows(df, index=True, header=True), start=1):
        for c_idx, value in enumerate(row, start=1):
            sheet.cell(row=r_idx, column=c_idx, value=value)

    # Define the red fill color
    red_fill = PatternFill(start_color='FF8989', end_color='FFFF0000', fill_type='solid')

    # Identifying p-value columns by their headers
    p_value_columns = [idx for idx, col in enumerate(df.columns) if 'P-value' in col]

    # Apply conditional formatting for each p-value column
    for col_idx in p_value_columns:
        for row in sheet.iter_rows(min_row=2, min_col=col_idx + 1, max_col=col_idx + 1):
            cell = row[0]  # There's only one cell in the row tuple
            try:
                # Check if the cell value is a float and less than 0.05
                if float(cell.value) < 0.05:
                    cell.fill = red_fill
            except (ValueError, TypeError):
                # Ignore cells that are not numeric
                continue

    # Save the modified workbook
    wb.save(output_file_name)

SAVE_ANOVA_TABLE_FILE = f"{parent}/twoWayANOVA_tables/{PERTURBATION}/{plate_type}_{PERTURBATION}_ANOVAresults_basal_MAXDOSE.xlsx"
color_p_values(nf,SAVE_ANOVA_TABLE_FILE)

In [ ]:
def add_barlabels(ax, bar_width=0.35):
        # Iterate through the patches (bars) in the Axes object
        for bar in ax.patches:
                # The bar's width and location can be accessed through the bar's attributes
                x = bar.get_x() + bar.get_width() / 2
                y = bar.get_height()
                # Format the label with the desired precision here; round(y, 2) for two decimal places
                label = f"{y:.2f}"
                
                # Place the label above the bar, centered
                ax.text(x, y, label, ha='center', va='bottom', color='pink', fontstyle='italic', fontsize=6,
                        bbox=dict(facecolor='pink', alpha=0.3, edgecolor='none', pad=0))

In [ ]:
def add_slope_labels(ax, testparam_df, gene_slope_dict, bar_width=0.35):
    mutants = list(testparam_df['Gene'].unique())
    chemical_types = list(testparam_df['Chemical_Type'].unique())
    
    fig_width, fig_height = ax.figure.get_size_inches()
    numgenes_plot = range(len(mutants))
    eg_width = fig_width/ len(numgenes_plot)
    bar_width = ax.patches[0].get_width()
    space_genes = numgenes_plot + eg_width -1 - bar_width
    print(space_genes)
    
    bars = ax.patches
    y_values = [bar.get_height() for bar in bars]
    top_yval = max(y_values)
    for ind, gene in enumerate(mutants):
        ax.text(space_genes[ind], top_yval + 0.01,
                    f"Slope = {gene_slope_dict[gene]['slope']:.2f}\n"
                    f"P-value = {gene_slope_dict[gene]['p']:.2e}",
                    color='green', fontweight='bold', fontsize=6,
                    ha='center', va='bottom',
                    bbox=dict(facecolor='gray', alpha=0.5, edgecolor='none', pad=1))
    # # Iterate through genes and chemical types
    # for gene in genes:
    #     if gene not in gene_slope_dict:
    #         print(f"No slope data for gene: {gene}")
    #         continue

    #     # Find the bars corresponding to the current gene
    #     gene_bars = [bar for bar in ax.patches if bar.get_height() > 0 and 
    #                  plt.getp(bar, 'label').startswith(gene) and 
    #                  plt.getp(bar, 'label').endswith(chemical_types[0])]  # Assuming the first chemical type is present

    #     if not gene_bars:
    #         print(f"No bars found for gene: {gene}")
    #         continue

    #     # Calculate the center x position of the group of bars for each gene
    #     gene_center_x = np.mean([bar.get_x() + bar.get_width() / 2 for bar in gene_bars])

    #     # Check for finite x positions
    #     if np.isfinite(gene_center_x):
    #         # Add the text label for the slope
    #         ax.text(gene_center_x, y_max + y_shift,
    #                 f"Slope = {gene_slope_dict[gene]['slope']:.2f}\n"
    #                 f"P-value = {gene_slope_dict[gene]['p']:.2e}",
    #                 color='green', fontweight='bold', fontsize=14,
    #                 ha='center', va='bottom',
    #                 bbox=dict(facecolor='gray', alpha=0.5, edgecolor='none', pad=1))
    #         print(f"Label added for gene: {gene}")
    #     else:
    #         print(f"Non-finite x position for gene: {gene}")


In [ ]:
import itertools

def compute_slope(testparam_df):
    genes = list(testparam_df['Gene'].unique())
    print(genes)
    gene_data_dict = {gene: [] for gene in genes}
    gene_slope_dict = {gene: float for gene in genes}
    for gene, perturb in itertools.product(testparam_df['Gene'].unique(), testparam_df['Chemical_Type'].unique()):
        mask = testparam_df['Gene'].str.contains(gene) & testparam_df['Chemical_Type'].str.contains(perturb)
        gene_data_dict[gene] = testparam_df[mask]['Value'].tolist()
        slope, intercept, r, p, se = linregress(np.arange(len(gene_data_dict[gene])), gene_data_dict[gene])
        gene_slope_dict[gene] = {"slope":slope, "r":r, "p":p, "se":se}
    return gene_slope_dict

trial = compute_slope(TestParamsSectioned_dict['Network Burst Duration - Avg (sec)'])

KeyError: 'Gene'

In [ ]:
TestParamsSectioned_dict['Network Burst Duration - Avg (sec)']

{'GPR37L_1':     Chemical_Type      Gene   Value
 0           basal  GPR37L_1 5.2e-01
 1           basal  GPR37L_1 4.6e-01
 2           basal  GPR37L_1 4.8e-01
 3           basal  GPR37L_1 2.7e-01
 4           basal  GPR37L_1 3.5e-01
 ..            ...       ...     ...
 955      CNO_20uM      unt2 7.9e-01
 956      CNO_20uM      unt2 4.1e-01
 957      CNO_20uM      unt2 2.9e-01
 958      CNO_20uM      unt2 3.4e-01
 959      CNO_20uM      unt2 3.8e-01
 
 [240 rows x 3 columns],
 'GPR37L_2':     Chemical_Type      Gene   Value
 120         basal  GPR37L_2 2.8e-01
 121         basal  GPR37L_2 6.1e-01
 122         basal  GPR37L_2 3.1e-01
 123         basal  GPR37L_2 2.9e-01
 124         basal  GPR37L_2 6.9e-01
 ..            ...       ...     ...
 955      CNO_20uM      unt2 7.9e-01
 956      CNO_20uM      unt2 4.1e-01
 957      CNO_20uM      unt2 2.9e-01
 958      CNO_20uM      unt2 3.4e-01
 959      CNO_20uM      unt2 3.8e-01
 
 [240 rows x 3 columns],
 'LGI2_A12':     Chemical_Type    

In [28]:
def create_bar_chart(testparameter, df, show_barlabels = True, control_var="unt2"):
    bar_width=0.20
    plt.figure(figsize=(10, 6)) # Adjust the size as needed
    # Create a bar plot
    ax = sns.barplot(x='Gene', y='Value', hue='Chemical_Type', data=df, errorbar='se')

    plt.title(f"{testparameter}")
    plt.xlabel('Gene')
    plt.ylabel('Mean of Replicates')
    plt.legend(title='Chemical Type')
    if show_barlabels:
        add_barlabels(ax)
    gene_slope_dict = compute_slope(df)
    print(f"GENE_SLOPE_dICT: {gene_slope_dict}")
    if show_barlabels:
        add_barlabels(ax)  # This function needs to be defined elsewhere
    add_slope_labels(ax, df, gene_slope_dict, bar_width)  # Adding slope labels to the plot
    plt.tight_layout()
    save_figfile = f"ANOVA_plots/{PERTURBATION}/{plate_type}_2wayANOVAtest/{testparameter}"
    ensure_dirs_exists(save_figfile)
    plt.savefig(save_figfile)
    plt.show()
    return

for test_param, table in TestParams_dict.items():
    if table['Chemical_Type'].nunique() > 1:
        create_bar_chart(test_param, table)

ValueError: Could not interpret input 'Gene'

<Figure size 1000x600 with 0 Axes>

In [29]:
def calculate_Tukeys(testparameter, df, show_barlabels = False, control_var="unt2"):  
    ## Perform Tukey's HSD test
    tukey = pairwise_tukeyhsd(endog=df['Value'],
                            groups=df['Gene'] + '_' + df['Chemical_Type'],
                            alpha=0.05)
    tukey_df = pd.DataFrame(data=tukey.summary().data[1:], columns=tukey.summary().data[0])
    tukey_control = tukey_df[tukey_df['group1'].str.contains('unt2') | tukey_df['group2'].str.contains('unt2')]
    mask = tukey_control['group1'].str.split('_').str[-1] == tukey_control['group2'].str.split('_').str[-1]
    matching_perturb_rows = tukey_control[mask]
    return matching_perturb_rows

In [ ]:
0101010
1
0101010
0
0101010